In [1]:
import csv
import numpy as np
from pprint import pprint
from sturdr.nav.ephemeris import GetNavStates
from sturdr.nav.estimation import LeastSquares, LeastSquaresPos
from sturdr.utils.constants import LIGHT_SPEED, GPS_L1CA_CARRIER_FREQ, GPS_L1CA_CODE_FREQ
from sturdr.utils.coordinates import ecef2lla, eci2ecef
from sturdr.dsp.discriminator import DllVariance

BETA = LIGHT_SPEED / GPS_L1CA_CODE_FREQ
LAMBDA = LIGHT_SPEED / GPS_L1CA_CARRIER_FREQ

# PRN = [1, 7, 14, 17, 19, 21, 30]
CNo = 10**(np.asarray([36.826, 41.229, 43.359, 36.599, 36.312, 43.221, 42.617])/10)
# ToW = 413658.0
# code_phase = np.array([11421.60386881124, 8209.110011406465, 11324.089926379798, 10902.806345747738, 3954.6763081617355, 4235.6660033346, 12222.95770424594])
ToW = 413660.94
code_phase = np.array([11425.045418898622, 8208.204583024342, 11330.938206410516, 10909.064860703384, 3964.2325959784534, 4235.030732526724, 12225.865036006284])
code_phase_time = code_phase / GPS_L1CA_CODE_FREQ
transmit_time = ToW + code_phase_time
receive_time = transmit_time.max() + 0.068802 # common receive time/sample including transmit time
tgd = np.zeros(7)
ephem = {}

with open("../results/GPS_L1CA_TEST_Ephemeris.csv", "r") as file:
    reader = csv.DictReader(file)
    for i,item in enumerate(reader):
        item.pop('id', None)
        item.pop('iode', None)
        item.pop('iodc', None)
        item.pop('ura', None)
        item.pop('health', None)
        for k,v in item.items():
            item[k] = np.float64(v)
        item["transmit_time"] = transmit_time[i]
        ephem[i] = item
# pprint(ephem[0])
# pprint(ephem[0].keys())

pprint(code_phase_time)
pprint(transmit_time)

array([0.01116818, 0.00802366, 0.01107619, 0.0106638 , 0.00387511,
       0.00413981, 0.01195099])
array([413660.95116818, 413660.94802366, 413660.95107619, 413660.9506638 ,
       413660.94387511, 413660.94413981, 413660.95195099])


In [2]:
sv_pos = np.zeros((7,3))
sv_pos_new = np.zeros((7,3))
sv_vel = np.zeros((7,3))
sv_clk = np.zeros((7,3))

# get satellite positions based on transmit time
for i in range(7):
    sv_clk[i,:], sv_pos[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    # sv_pos_new[i,:] = CorrectEarthRotation(-(receive_time - transmit_time[i]), sv_pos[i,:])
    tgd[i] = ephem[i]['tgd']
# pprint(sv_pos - sv_pos_new)

psr = (receive_time - transmit_time + sv_clk[:,0] + tgd)

# get corrected positions with new transmit time
# new_transmit_time = receive_time - psr
for i in range(7):
    # ephem[i]['transmit_time'] = new_transmit_time[i]
    # sv_clk[i,:], sv_pos_new[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    # sv_pos_new[i,:] = eci2ecef(+psr[i], sv_pos_new[i,:])
    sv_pos_new[i,:] = eci2ecef(+psr[i], sv_pos[i,:])

# pprint(transmit_time - new_transmit_time)
pprint(sv_pos - sv_pos_new)

array([[ 84.2316896 ,  67.84338932,   0.        ],
       [133.94239717,  35.31528757,   0.        ],
       [ 79.33444347,  -8.60836604,   0.        ],
       [109.54888063, -51.31422675,   0.        ],
       [113.81410416, -96.80419314,   0.        ],
       [ 41.4330354 ,  85.07465365,   0.        ],
       [124.32760305,  -9.59882249,   0.        ]])


In [3]:
R = np.diag(BETA**2 * DllVariance(CNo, 0.02))
x, _, _ = LeastSquaresPos(sv_pos_new, psr * LIGHT_SPEED, R, np.zeros(4))
true_x = np.asarray([422596.629, -5362864.287, 3415493.797])
true_range = np.linalg.norm(true_x[...] - sv_pos_new, axis=1)

print(f"lla_est = {np.array2string(ecef2lla(x[:3]), precision=8)}")
print(f"x_est   = {np.array2string(x, precision=3)}")
print(f"x_true  = {np.array2string(true_x, precision=3)}")
print(f"diff    = {np.array2string(true_x - x[:3], precision=3)}, norm = {np.array2string(np.linalg.norm(true_x - x[:3]), precision=6)}")
print(f"psr     = {np.array2string(psr*LIGHT_SPEED, precision=0)}")
print(f"r_true  = {np.array2string(true_range, precision=0)}")
print(f"diff    = {np.array2string(true_range - psr*LIGHT_SPEED, precision=3)}")

lla_est = [ 32.58631296 -85.49441895 204.2527693 ]
x_est   = [  422592.687 -5362870.509  3415502.064    67791.614]
x_true  = [  422596.629 -5362864.287  3415493.797]
diff    = [ 3.942  6.222 -8.267], norm = 11.072759
psr     = [20921615. 21870202. 20887920. 21223799. 23134619. 23014302. 20468445.]
r_true  = [20853828. 21802416. 20820141. 21156019. 23066833. 22946518. 20400658.]
diff    = [-67787.691 -67786.152 -67779.027 -67779.933 -67785.989 -67784.498
 -67786.126]
